<a href="https://colab.research.google.com/github/PreethamNaik12/StairNet/blob/main/Copy_of_LegendaryNote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3 smart_open

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.8 MB/s eta 0:00:00


In [1]:
# import boto3
from smart_open import open
import zipfile
import io
import os
import glob
import shutil
import tempfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from PIL import Image
from torch.utils.data import Dataset
from collections import defaultdict
from torchvision import transforms
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from google.colab import drive

In [3]:
# Set up AWS credentials (replace with your own credentials or set them up in environment variables)
from google.colab import userdata

AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')
S3_BUCKET = userdata.get('S3_BUCKET')
S3_KEY = userdata.get('S3_KEY')

In [4]:
# Set environment variables for AWS credentials
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')

In [5]:


# Create a session using the credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# Connect to S3
s3 = session.resource('s3')

# Define the file name for the downloaded zip file
local_zip_path = '/content/downloaded_file.zip'

# Download the zip file from S3
s3.Bucket(S3_BUCKET).download_file(S3_KEY, local_zip_path)
print(f'File {S3_KEY} downloaded from bucket {S3_BUCKET} to {local_zip_path}')



File data/7540/StairNet.zip downloaded from bucket ieee-dataport to /content/downloaded_file.zip


In [7]:
import shutil

# Source file path in Colab
zip_file_path = '/content/downloaded_file.zip'

# Destination path in Google Drive
drive_path = '/content/drive/MyDrive/myfile.zip'

# Copy the zip file
shutil.copy(zip_file_path, drive_path)


'/content/drive/MyDrive/myfile.zip'

In [17]:
import zipfile
import io
import os

# Path to the zip file in Google Drive
zip_file_path = '/content/downloaded_file.zip'

# Load the zip file into memory
with open(zip_file_path, 'rb') as f:
    zip_data = f.read()

# Create a BytesIO object from the zip file data
zip_bytes_io = io.BytesIO(zip_data)

# Create a ZipFile object from the BytesIO object
with zipfile.ZipFile(zip_bytes_io, 'r') as zip_ref:
    # List all files in the zip archive
    zip_file_list = zip_ref.namelist()

    # Extract files into RAM
    extracted_files = {}
    for file_name in zip_file_list:
        with zip_ref.open(file_name) as file:
            extracted_files[file_name] = file.read()

# Now extracted_files is a dictionary with file names as keys and file data as values


In [22]:
extracted_files["StarirNet/ISLG/"]

b''

In [6]:
# Unzip the file
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/unzipped_files')
    print(f'Files extracted to /content/unzipped_files')

# List extracted files
extracted_files = os.listdir('/content/unzipped_files')
print(f'Extracted files: {extracted_files}')


Files extracted to /content/unzipped_files
Extracted files: ['StairNet']


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the file
file_path = '/content/drive/My Drive/ColabData/dataset/images'

# Check if the file exists
if os.path.exists(file_path):
    print(f"File '{file_path}' exists.")
else:
    print(f"File '{file_path}' does not exist.")


Mounted at /content/drive
File '/content/drive/My Drive/ColabData/dataset/images' exists.


In [ ]:
import glob

def get_filenames(path):
    return [os.path.basename(f) for f in glob.glob(os.path.join(path, '*'))]

all_images_and_labels = get_filenames(file_path)

In [ ]:
def get_filenames(path):
    with os.scandir(path) as entries:
        for entry in entries:
            yield entry.name

all_images_and_labels = list(get_filenames(file_path))


OSError: [Errno 5] Input/output error: '/content/drive/My Drive/ColabData/dataset/images'

In [ ]:
# Define the directory containing the images
image_directory = '/content/images'

# Initialize counters for each suffix
lg_count = 0
lg_is_count = 0
is_count = 0
is_lg_count = 0

# List all files in the directory
for filename in all_images_and_labels:
    # Check and count for each suffix
    if filename.endswith(' LG.jpg'):
        lg_count += 1
    elif filename.endswith(' LGIS.jpg'):
        lg_is_count += 1
    elif filename.endswith(' IS.jpg'):
        is_count += 1
    elif filename.endswith(' ISLG.jpg'):
        is_lg_count += 1

# Print the counts
print(f"Number of images ending with 'LG': {lg_count}")
print(f"Number of images ending with 'LG-IS': {lg_is_count}")
print(f"Number of images ending with 'IS': {is_count}")
print(f"Number of images ending with 'IS-LG': {is_lg_count}")


Number of images ending with 'LG': 0
Number of images ending with 'LG-IS': 0
Number of images ending with 'IS': 0
Number of images ending with 'IS-LG': 0


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(42)
np.random.seed(42)

Using device: cuda


## Step 2:

- First, let's define the dataset structure

dataset_structure = {
    'IS': 48179,
    'ISLG': 9025,
    'LG': 442360,
    'LGIS': 15888
}

total_samples = sum(dataset_structure.values())
print(f"Total samples: {total_samples}")

- Now, let's create functions to split the data according to the given percentages:

In [ ]:
# def split_dataset(dataset_structure, train_pct=0.895, val_pct=0.035, test_pct=0.07):
#     train_set, val_set, test_set = {}, {}, {}

#     for class_name, total_samples in dataset_structure.items():
#         train_samples = int(total_samples * train_pct)
#         val_samples = int(total_samples * val_pct)
#         test_samples = total_samples - train_samples - val_samples

#         train_set[class_name] = train_samples
#         val_set[class_name] = val_samples
#         test_set[class_name] = test_samples

#     return train_set, val_set, test_set

# train_set, val_set, test_set = split_dataset(dataset_structure)

# print("Training set:", train_set)
# print("Validation set:", val_set)
# print("Test set:", test_set)

In [14]:
def split_data(file_list, train_pct=0.895, val_pct=0.035, test_pct=0.07):
    random.shuffle(file_list)
    total = len(file_list)
    train_end = int(total * train_pct)
    val_end = train_end + int(total * val_pct)

    return file_list[:train_end], file_list[train_end:val_end], file_list[val_end:]

- For loading the data, we'll need to create a custom dataset class

In [21]:
class StairNetDataset(Dataset):
    def __init__(self, root_dir, file_list, transform=None):
        self.root_dir = root_dir
        self.file_list = file_list
        self.transform = transform
        self.class_to_idx = {'IS': 0, 'ISLG': 1, 'LG': 2, 'LGIS': 3}

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        # Extract label from filename
        parts = img_name.split()
        label = parts[-1].split('.')[0]  # Get the last part before the file extension

        # Handle the case where the label might be two words (e.g., 'LG IS')
        if len(parts) > 2 and parts[-2] in ['LG', 'IS']:
            label = f"{parts[-2]} {label}"

        label_idx = self.class_to_idx[label]

        if self.transform:
            image = self.transform(image)

        return image, label_idx

- creat a stratified dataset
- setup the dataloaders

In [3]:
root_dir = '/content/drive/My Drive/ColabData/dataset/images'

In [8]:

# Define root directory for images and list all files
all_files = os.listdir(root_dir)
image_files = [f for f in all_images_and_labels if f.endswith('.jpg')]

# Define the path to save the text file in Google Drive
output_path = '/content/drive/My Drive/image_files.txt'

# Save the list to a text file in Google Drive
with open(output_path, 'w') as file:
    for image_file in image_files:
        file.write(f"{image_file}\n")

print(f"File saved to {output_path}")


OSError: [Errno 5] Input/output error: '/content/drive/My Drive/ColabData/dataset/images'

In [23]:
def split_data_stratified(file_list, train_pct=0.895, val_pct=0.035, test_pct=0.07):
    # Group files by class
    class_files = defaultdict(list)
    for file in file_list:
        parts = file.split()
        label = parts[-1].split('.')[0]
        if len(parts) > 2 and parts[-2] in ['LG', 'IS']:
            label = f"{parts[-2]} {label}"
        class_files[label].append(file)

    train_files, val_files, test_files = [], [], []

    for class_label, files in class_files.items():
        random.shuffle(files)
        n_files = len(files)
        n_train = int(n_files * train_pct)
        n_val = int(n_files * val_pct)

        train_files.extend(files[:n_train])
        val_files.extend(files[n_train:n_train+n_val])
        test_files.extend(files[n_train+n_val:])

    random.shuffle(train_files)
    random.shuffle(val_files)
    random.shuffle(test_files)

    return train_files, val_files, test_files

# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Set up data
root_dir = '/content/all_images'
all_files = os.listdir(root_dir)
image_files = [f for f in all_files if f.endswith('.jpg')]

# Split data
train_files, val_files, test_files = split_data_stratified(image_files)

# Create datasets
train_dataset = StairNetDataset(root_dir, train_files, transform=train_transform)
val_dataset = StairNetDataset(root_dir, val_files, transform=val_transform)
test_dataset = StairNetDataset(root_dir, test_files, transform=val_transform)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 17896
Number of validation samples: 696
Number of test samples: 1404


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


- check the distribution (caution: takes very very long time to count! so run this if you have a lot of time availale)

In [ ]:
## Print class distribution in each split

# def print_class_distribution(dataset):
#     class_counts = defaultdict(int)
#     for _, label in dataset:
#         class_counts[label] += 1
#     total = sum(class_counts.values())
#     for label, count in class_counts.items():
#         print(f"Class {label}: {count} ({count/total:.2%})")

# print("\nTraining set class distribution:")
# print_class_distribution(train_dataset)
# print("\nValidation set class distribution:")
# print_class_distribution(val_dataset)
# print("\nTest set class distribution:")
# print_class_distribution(test_dataset)

In [24]:
class CustomMobileNetV2(nn.Module):
    def __init__(self, num_classes=4, dropout_rate=0.2):
        super(CustomMobileNetV2, self).__init__()

        # Load pretrained MobileNetV2
        self.model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)

        # Freeze first 5 layers
        for i, param in enumerate(self.model.features[:5].parameters()):
            param.requires_grad = False

        # Modify the classifier
        in_features = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [25]:
# !pip install torchsummary

In [26]:
# Create the model
model = CustomMobileNetV2(num_classes=4, dropout_rate=0.2)

# Print model summary
from torchsummary import summary
summary(model, (3, 224, 224), device='cuda')

# Verify number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 65.0MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# # for optimization
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.1)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.to(device)
    best_val_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        #optimization
        # Add this line to update the scheduler
#         scheduler.step(val_loss)

        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')

    print(f'Best validation accuracy: {best_val_accuracy:.4f}')

# Use the function
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10: 100%|██████████| 140/140 [03:19<00:00,  1.42s/it]


Epoch 1/10:
Train Loss: 0.6141, Train Accuracy: 0.7598
Val Loss: 0.3509, Val Accuracy: 0.8563


Epoch 2/10: 100%|██████████| 140/140 [03:14<00:00,  1.39s/it]


Epoch 2/10:
Train Loss: 0.4740, Train Accuracy: 0.8200
Val Loss: 0.2929, Val Accuracy: 0.8865


Epoch 3/10: 100%|██████████| 140/140 [03:12<00:00,  1.38s/it]


Epoch 3/10:
Train Loss: 0.4188, Train Accuracy: 0.8423
Val Loss: 0.2824, Val Accuracy: 0.9023


Epoch 4/10: 100%|██████████| 140/140 [03:18<00:00,  1.42s/it]


Epoch 4/10:
Train Loss: 0.3938, Train Accuracy: 0.8517
Val Loss: 0.2702, Val Accuracy: 0.9052


Epoch 5/10:   6%|▌         | 8/140 [00:11<02:02,  1.08it/s]

### New way of dataloading

In [4]:
zip_file_path = '/content/drive/MyDrive/myfile.zip'

### make folders from zip file

In [12]:
# Destination directory on Colab disk
destination_path = '/content/LGIS'

# Create the destination directory if it doesn't exist
os.makedirs(destination_path, exist_ok=True)

# List files in the specified folder within the zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    all_files = zip_ref.namelist()
    # Filter files to only include those in 'StairNet/IS/'
    target_files = [f for f in all_files if f.startswith('StairNet/LGIS/')]

    # Limit to the first all files
    target_files = target_files[:5000]

    # Extract the files
    for file in target_files:
        zip_ref.extract(file, destination_path)

In [13]:
import os

# Specify the path to the folder
folder_path = '/content/LGIS'

# Initialize a counter for files
file_count = 0

# Iterate over files in the folder
for _, _, files in os.walk(folder_path):
    file_count += len(files)

print(f"Number of files in '{folder_path}': {file_count}")


Number of files in '/content/LGIS': 4999


In [16]:
os.makedirs('all_images', exist_ok=True)

In [20]:
# Source and destination folder paths
source_folder = '/content/LGIS/StairNet/LGIS'
destination_folder = '/content/all_images'

# Check if the source folder exists
if os.path.exists(source_folder):
    # Iterate over files in the source folder
    for file_name in os.listdir(source_folder):
        # Construct full file paths
        source_file = os.path.join(source_folder, file_name)
        destination_file = os.path.join(destination_folder, file_name)

        # Check if the current path is a file
        if os.path.isfile(source_file):
            # Copy the file to the destination folder
            shutil.copy(source_file, destination_file)
            print(f"File '{file_name}' copied from {source_folder} to {destination_folder}")
else:
    print(f"Source folder '{source_folder}' does not exist.")


File '['IMG_02_1'] frame 18432 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_04_4'] frame 23268 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_03_2'] frame 24618 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_03_3'] frame 37650 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_06_1'] frame 81126 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_02_3'] frame 88926 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_03_4'] frame 15876 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_02_3'] frame 21504 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_09_1'] frame 38880 LGIS.jpg' copied from /content/LGIS/StairNet/LGIS to /content/all_images
File '['IMG_05_1'] frame 42858 LGIS.jpg' copied from /content/LG